# Point Regressor Surrogate Models
Eamon Whalen

In [1]:
import sys
import numpy as np
import pandas as pd
import altair as alt

from gcnSurrogate.models.pointRegressorSurrogateModel import PointRegressor
from gcnSurrogate.readers.loadGhGraphs import loadGhGraphs
from gcnSurrogate.visualization.altTrussViz import plotTruss, interactiveErrorPlot
from gcnSurrogate.util.gcnSurrogateUtil import *

## 1. Load simulation data

In [4]:
doeFile = "data/2D_Truss_v1.3/gh/design_7_N_1000.csv"
allGraphsUnfiltered = loadGhGraphs(doeFile, NUM_DV=5)

maxes = [max(np.abs(graph.y.numpy().flatten())) for graph in allGraphsUnfiltered]
source = pd.DataFrame(maxes, columns=['maxes'])
source.describe()

,maxes
count,1000.000000
mean,0.199516
std,2.524026
min,0.006570
25%,0.015425
50%,0.024334
75%,0.046981
max,67.325867


In [5]:
plotTruss(allGraphsUnfiltered[0], showDeformed=True, defScale=100)

alt.LayerChart(...)

## 2. Filter and partition

In [6]:
allGraphs = filterbyDisp(allGraphsUnfiltered, 0.9)
trainData, valData, testData = partitionGraphList(allGraphs)

maxes = [max(np.abs(graph.y.numpy().flatten())) for graph in allGraphs]
source = pd.DataFrame(maxes, columns=['maxes'])
source.describe()

,maxes
count,900.000000
mean,0.028952
std,0.019071
min,0.006570
25%,0.014951
50%,0.021719
75%,0.038367
max,0.097861


## 3. Train and test random forest point regressors

In [7]:
rf = PointRegressor('Random Forest')
rf.trainModel(trainData, valData, 
             flatten=False, 
             logTrans=False, 
             ssTrans=True, 
             saveDir='results/rf01/')

trainRes = rf.testModel(trainData)
testRes = rf.testModel(testData)
pd.DataFrame([trainRes, testRes], index=['train', 'test'])

trained 30 random forest models in 13.49 seconds


,mse,mae,mre,peakR2,maxAggR2,meanAggR2,minAggR2
train,0.000004,0.001045,0.043436,0.955418,0.999193,0.981765,0.962601
test,0.000021,0.002509,0.101468,0.684708,0.996946,0.876139,0.741907


## 4. Visualize some predictions

In [8]:
i = 8
pred = rf.predict([testData[i]])[0]
plotTruss(testData[i], showDeformed=True, defScale=100, prediction=pred)

alt.LayerChart(...)

In [9]:
# interactive scatter plot
alt.data_transformers.enable('json')
allPreds = rf.predict(testData)
display(interactiveErrorPlot(testData, allPreds))
alt.data_transformers.enable('default');

alt.HConcatChart(...)